In [31]:
from carto.auth import APIKeyAuthClient
from carto.sql import SQLClient
from carto.exceptions import CartoException
from elasticsearch import Elasticsearch
import requests
import json

### Auxiliar function

## authentication

In [4]:
# For unauthenticated requests, we need to create a NoAuthClient object:
# USERNAME="hpclab"
# USR_BASE_URL = "https://{user}.carto.com/".format(user=USERNAME)
# auth_client = NoAuthClient(base_url=USR_BASE_URL)

In [8]:
# For API key authenticated requests, we need to create an APIKeyAuthClient instance:
USERNAME="hpclab"
APIKEY = "e97d6bbf73f61f1faa5a3bafb81c370bdc131de5"
USR_BASE_URL = "https://{user}.carto.com/".format(user=USERNAME)
auth_client = APIKeyAuthClient(api_key=APIKEY, base_url=USR_BASE_URL)


In [99]:
## common queries 
# query all the data
query_all={"query": {"match_all": {}}}

## ES - retrieval

In [182]:
str_a = "I'm"
str_a.replace('\'',  " ")


'I m'

In [184]:
# es_url = 'test.tripbuilder.isti.cnr.it'
# es_port = 9200
# sync_cycle = 10

# # query last hour
# query_last_hour={    
#     "query": {
#         "range" : {
#             "created_at" : {
#                 "gte" : "now-1h"                
#             }
#         }
#     }
# }

# # search on es
# es = Elasticsearch([es_url], http_auth=('elastic', 'changeme'), port=es_port)
# response = es.search(index="stream", doc_type='tweet_snippet', size=100
#                      , body=query_last_hour
#                      , sort={"created_at": {"order" : "asc"}})

# # cleaning results
# tweets = [x['_source'] for x in response['hits']['hits']]

# text as unicode
for i in range(0, len(tweets)):
    # tweets[i]['text'] = tweets[i]['text'].encode('utf-8')
    tweets[i]['text'] = tweets[i]['text'].replace('\'',  " ")   
    
print "tweets", len(tweets)    

tweets 100


## sql api

In [107]:
# select
sql = SQLClient(auth_client)

try:
    data = sql.send('select * from test_table')
except CartoException as e:
    print("some error ocurred", e)

print data['rows']

[{u'the_geom_webmercator': None, u'cartodb_id': 2, u'the_geom': None, u'name': u'str', u'description': u'str'}, {u'the_geom_webmercator': None, u'cartodb_id': 1, u'the_geom': None, u'name': u'test', u'description': u'ops'}, {u'the_geom_webmercator': u'0101000020110F00007AAC558CFF2C5EC12EB177BA36925741', u'cartodb_id': 3, u'the_geom': u'0101000020E61000003CDBA337DCC351C06D37C1374D374840', u'name': u'str', u'description': u'str'}, {u'the_geom_webmercator': u'0101000020110F00007AAC558CFF2C5EC12EB177BA36925741', u'cartodb_id': 4, u'the_geom': u'0101000020E61000003CDBA337DCC351C06D37C1374D374840', u'name': u'str', u'description': u'str'}, {u'the_geom_webmercator': u'0101000020110F0000AC881E18CAAE55C1F9D630B76FEF5341', u'cartodb_id': 5, u'the_geom': u'0101000020E610000078B6476FB88749C06D37C1374D374540', u'name': u'str', u'description': u'str'}]


In [151]:
def centroid(points):    
    x_coords = [p[0] for p in points]
    y_coords = [p[1] for p in points]   
    _len = len(points)
    centroid_x = sum(x_coords)/_len
    centroid_y = sum(y_coords)/_len
    return [centroid_x, centroid_y]

In [153]:
print tweets[2]['bounding_box']['coordinates'][0]
print centroid(tweets[2]['bounding_box']['coordinates'][0])


# print len(tweets[2]['bounding_box']['coordinates'][0])


# sudo apt-get install libgeos-dev
# sudo yum install geos-devel

print tweets[2].keys()


[[-76.889372, 39.076292], [-76.889372, 39.115394], [-76.827105, 39.115394], [-76.827105, 39.076292]]
[-76.8582385, 39.095843]
[u'username', u'text', u'created_at', u'coordinates', u'id', u'bounding_box', u'img_category', u'day', u'media_url']


In [166]:
print len(tweets)

100


In [188]:
# insert

import random
import time

# while True:
#     print random.randint(0, 2)
#     time.sleep(1)


sql = SQLClient(auth_client)
sql_str = ''
for tweet in tweets:    
    try:    
        coordinates =  tweet['coordinates'] if tweet['coordinates'] is not None else centroid(tweet['bounding_box']['coordinates'][0])
        the_geom = "ST_GeomFromText('POINT({lng} {lat})', 4326)".format(lng=coordinates[0], lat= coordinates[1])
        
        insert_value = "{the_geom}, '{username}', '{img_category}', '{text}', {id}, '{media_url}', {day}".format(
            the_geom=the_geom, username=tweet['username'], img_category=tweet['img_category'], text=tweet['text'], id=tweet['id'], media_url=tweet['media_url'], day=tweet['day'])        
        
        sql_str = "insert into tweets_stream (the_geom, username, img_category, text, id, media_url, day) values ({insert_value});".format(insert_value=insert_value)    
        # print sql_str
        sql.send(sql_str)
        
        # time.sleep(15)
        
    except CartoException as e:
        print "some error ocurred", e, sql_str
        
# print data['rows']